In [ ]:
import pandas as pd
from plotnine import *

checkv = pd.read_csv(
    str(snakemake.input)
    # '/home/carsonjm/CarsonJM/phide_piper/results/06_VIRUS_QUALITY/01_checkv/quality_summary.tsv'
    , sep='\t')

checkv.loc[checkv['completeness'] > 90, 'high-quality'] = True
checkv.loc[checkv['completeness'] > 50, 'medium-quality'] = True
checkv.loc[checkv['completeness'].notnull(), 'low-quality'] = True
checkv.loc[checkv['completeness'].isnull(), 'undetermined'] = True
checkv.loc[(checkv['completeness'].isnull() | checkv['completeness'].notnull()), 'total'] = True
 
checkv_melt = checkv.melt(id_vars='contig_id', value_vars=['high-quality', 'medium-quality', 'low-quality', 'undetermined', 'total'])
checkv_melt2 = checkv_melt[checkv_melt['value'] == True]
checkv_melt2_rn = checkv_melt2.rename(columns={'variable':'quality'})
checkv_melt2_rn['quality'] = pd.Categorical(checkv_melt2_rn['quality'], categories=['total', 'undetermined', 'low-quality', 'medium-quality', 'high-quality'], ordered = True)

vq_plot = (
    ggplot(checkv_melt2_rn)
    + geom_bar(aes(x='quality'), position="dodge")
    + ylab("Number of putative viruses identified")
    + xlab("CheckV virus quality categories")
)

vq_plot.save(str(snakemake.output.figure), dpi=600)